In [4]:
import os

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches

import numpy as np
import pandas as pd
import geopandas as gpd

%matplotlib inline

datadir = r"./data/"

In [19]:
# load data
world_shapefile = os.path.join( datadir , "WMO/WMO-shapefile-BCT-ex.shp" )

gdf_world = gpd.read_file( world_shapefile )
gdf_world.crs = "epsg:4326"
gdf_world.set_index("POL_C_CODE",inplace=True)

print(gdf_world.shape)

df = pd.read_csv("GBON-HZ.csv",encoding="utf8").rename(columns={"Unnamed: 0":"country_id"}).set_index("country_id")
df=gdf_world[["STATUS","TERR_NAME","wmo_region","wmo_member","geometry"]].join(df[["HR_baseline",]],how="right")
df.head()

(214, 11)


,STATUS,TERR_NAME,wmo_region,wmo_member,geometry,HR_baseline
AFG,WMO Member,Afghanistan,II,1956-09-11,"POLYGON ((71.26610 37.98232, 71.25214 37.92794...",233.248794
AGO,WMO Member,Angola,I,1977-03-16,"MULTIPOLYGON (((13.99750 -5.84861, 14.08028 -5...",303.123911
AIA,WMO Member,Anguilla,IV,None,"POLYGON ((-63.15375 18.16528, -63.16778 18.164...",NaN
ALB,WMO Member,Albania,VI,1957-07-29,"POLYGON ((19.79396 42.48409, 19.82250 42.47194...",60.485911
AND,WMO Member,Andorra,VI,-0001-11-30,"POLYGON ((1.73861 42.61638, 1.78172 42.56996, ...",NaN


In [ ]:
# select members to print
idx_member = df.STATUS == "WMO Member"
wmo_members = df[idx_member].index.values    
  
column="HR_baseline" #column containing the data
idx_missing = ( df[column].isna() & idx_member ) 
    
# plot config
missing = {   "color": "lightgrey", "edgecolor": "red",
               "hatch": "///", "label": "Missing values" }

    
cmap="coolwarm"
norm = colors.TwoSlopeNorm(vmin=0, vcenter=200., vmax=1000)    
cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

fig,ax=plt.subplots(figsize=(15,15))

# plot countries
df[idx_member].to_crs("+proj=eqearth").plot(column=column, legend=False, cmap=cmap, norm=norm, ax=ax )
df[~df.index.isin(wmo_members)].to_crs("+proj=eqearth").plot(ax=ax, **missing) #plot non Members
df[idx_missing].to_crs("+proj=eqearth").plot(facecolor="grey",ax=ax)


# configure plot
ax.set_title(title, fontsize=15)
ticks=[0,50,100,200,600,1000]
ax_cbar = fig.colorbar( cbar , ax=ax, ticks=ticks)
ax_cbar.set_label('Horizontal resolution in km^2')
ax_cbar.ax.set_yticklabels(["0","50","100","200 (GBON)","600",">1000"])

ax.axis("off")